In [ ]:
#подключаем нужные нам библиотеки 
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
import binascii
import pdfkit
#получаем нужные данные
r = requests.get("https://video.ittensive.com/python-advanced/data-107235-2019-12-02.utf.json")
#формируем набор данных только из колонок "CourseName", "CoursesTimetable", "NameOfPark"
#именно эти колонки нужны нам для вывода в отчёте
data = pd.DataFrame(json.loads(r.content),
                   columns=["CourseName", "CoursesTimetable", "NameOfPark"])
#получаем название парка из комбинированной серии, получаем значение value словоря
data["NameOfPark"] = data["NameOfPark"].apply(lambda x: x["value"])
#переиминовываем колонки  для отчёта, 
data.columns = ["Активность", "Расписание", "Парк"]
#для ответа на поставленный вопрос найдём активность Тайцзицюань, 
#и выведем число записей с этой активностью
print ("Тайцзицюань: ",
       data[data["Активность"].str.contains("Тайцзицюань")]["Активность"].count())
#формируем диаграмму активности по паркам
#создаём холст
fig = plt.figure(figsize=(12,6))
area = fig.add_subplot(1,1,1)
#проводим группировку данных по парку, отсортируем эту группировку в порядке убывания
#чтобы взять первые 20 значений по посещаемости, и нанести их на круговую диаграмму
parks = data.groupby("Парк").count().sort_values("Активность", ascending=False)
parks.head(10)["Активность"].plot.pie(ax=area, label="")
#для временного хранения бинарных файлов изображения используем BytesIO()
#и его дальнейшего преобразование в base64 - формат
img = BytesIO()
plt.savefig(img)
#по факту у нас будет некоторый указатель в памяти, который мы используем
#для хранения данных изображения
#преобразовываем бинарные данные к base64, данные декодируем в UTF - 8
img = 'data:image/png;base64,' + binascii.b2a_base64(img.getvalue(),
                                    newline=False).decode("UTF-8")
#указываем максимальное количество символов в ячейках 
pd.set_option('display.max_colwidth', 1000)
#Формируем html - отчёт, в отчёте выводим изображение, уровень активности парка 
# и расписание 
html = '''<html>
<head>
    <title>Активности в парках Москвы</title>
    <meta charset="utf-8"/>
</head>
<body>
    <h1>Активности в парках Москвы</h1>
    <img src="E:/ittensive/assets/''' + img + '''" alt="Популярные парки"/>
    ''' + data.to_html(index=False) + '''
</body>
</html>'''
#генерируем PDF - файл, из html - строки, используя pdfkit
config = pdfkit.configuration(wkhtmltopdf='E:/programs/wk_html_pdf/wkhtmltopdf/bin/wkhtmltopdf.exe')
#задём параметры PDF - страницы
options = {
    'page-size': 'A4',
    'header-right': '[page]'
}
#сохраняем полученный результат в файл parks.pdf
pdfkit.from_string(html, 'E:/ittensive/assets/parks.pdf',
                   configuration=config, options=options)
with open("E:/ittensive/assets/parks.html", "w", encoding="utf-8") as file:
    file.write(html)